## AuxTel Image Header Check

In [ ]:
import logging
import re
import sys

from astropy.time import Time, TimeDelta
import numpy as np

import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient
from check_utils import check, check_float, check_near, check_not_empty, get_from_json

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
# TTS
dataPath="/repo/LATISS"
efd_name = "tucson_teststand_efd"
# RSP
#dataPath="/repo/main"
#efd_name = "ldf_stable_efd"

collections = ["LATISS/raw/all"]
instrument = "LATISS"
butler = dafButler.Butler(dataPath, collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [ ]:
obs_date = 20220418
seq_num = 57
dataId = {"instrument": instrument, "detector": 0,
          "exposure.day_obs": obs_date, "exposure.seq_num": seq_num}
raw = butler.get('raw.metadata', dataId)

In [ ]:
#header = raw.getInfo().getMetadata().toDict()
header = raw.toDict()
#print(list(header.keys()))
#print(header)

In [ ]:
if obs_date == 20220401 and seq_num == 43:
    header["DATE-BEG"] = "2022-04-01T23:26:47.45"
    header["DATE-END"] = "2022-04-01T23:26:52.435"
if obs_date == 20220401 and seq_num == 46:
    header["DATE-BEG"] = "2022-04-02T00:15:02.525"
    header["DATE-END"] = "2022-04-02T00:15:10.509"
if obs_date == 20220412 and seq_num == 2:
    header["DATE-BEG"] = "2022-04-12T18:30:27.202"
    header["DATE-END"] = "2022-04-12T18:30:28.225"
if obs_date == 20220418 and seq_num == 57:
    header["DATE-BEG"] = "2022-04-18T23:51:55.043"
    header["DATE-END"] = "2022-04-18T23:52:03.025"

In [ ]:
delta = TimeDelta(20.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai").utc
obs_end = Time(header["DATE-END"], format="fits", scale="tai").utc
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
#print(obs_beg)
#print(header["DATE-BEG"])
obs_id = header["OBSID"]
#print(f"IMGTYPE: {header['IMGTYPE']}")
#print(f"EXPTIME: {header['EXPTIME']}")
#print(f"DARKTIME: {header['DARKTIME']}")
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.ATCamera.logevent_startIntegration", "*",
                                               obs_beg, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.ATCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
# Scales are UTC because incoming timestamps are already TAI and astropy screws them up if the scale is TAI.
# This was fixed in astopy 5.
date_obs = Time(end_image_tel["timestampDateObs"][0], format="unix_tai", scale="tai")
date_end = Time(end_image_tel["timestampDateEnd"][0], format="unix_tai", scale="tai")
date_start_int = Time(start_int["private_sndStamp"][0], format="unix_tai", scale="tai").utc
date_eoit = Time(end_image_tel["private_sndStamp"][0], format="unix_tai", scale="tai").utc
print(date_start_int.isot)
print(date_eoit.isot)

In [ ]:
check("DATE-BEG", header, date_obs.isot)
check("DATE-END", header, date_end.isot)

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
image_readout_params_df = await client.select_top_n("lsst.sal.ATCamera.logevent_imageReadoutParameters",
                                                    "*", 1,
                                                    time_cut=date_start_int.isot)

In [ ]:
check("TELCODE", header, start_int["imageSource"][0])
check("CONTRLLR", header, start_int["imageController"][0])
check("DAYOBS", header, start_int["imageDate"][0])
check("SEQNUM", header, start_int["imageNumber"][0])
additional_keys = start_int["additionalKeys"][0].split(":")
additional_values = re.split(r'(?<!\\):', start_int["additionalValues"][0])
print(additional_keys, additional_values)
check("GROUPID", header, additional_values[additional_keys.index("groupId")].replace("\\", ""))
check("IMGTYPE", header, additional_values[additional_keys.index("imageType")])
check("TESTTYPE", header, additional_values[additional_keys.index("testType")])
check("OBSANNOT", header, image_readout_params_df["daqAnnotation"][0])
check("CURINDEX", header, start_int["imageIndex"][0])
check("MAXINDEX", header, start_int["imagesInSequence"][0])
check_not_empty("PROGRAM", header)
check_not_empty("REASON", header)

In [ ]:
check_not_empty("DETSIZE", header)
check("OVERH", header, image_readout_params_df["overRows"][0])
check("OVERV", header, image_readout_params_df["overCols"][0])
check("PREH", header, image_readout_params_df["preCols"][0])

In [ ]:
shutter_motion_profile_df = await client.select_top_n("lsst.sal.ATCamera.logevent_shutterMotionProfile",
                                                      "*", 1,
                                                      time_cut=date_eoit.isot)

In [ ]:
check_float("EXPTIME", header, start_int["exposureTime"][0])
check_float("DARKTIME", header, end_image_tel["darkTime"][0])
check_float("SHUTTIME", header, shutter_motion_profile_df["measuredExposureTime"][0])

In [ ]:
current_target_df = await client.select_top_n("lsst.sal.ATPtg.logevent_currentTarget", "*", 1,
                                              time_cut=date_start_int.isot)
current_target_status_start_df = await client.select_top_n("lsst.sal.ATPtg.currentTargetStatus", "*", 1,
                                                           time_cut=date_start_int.isot)
current_target_status_end_df = await client.select_top_n("lsst.sal.ATPtg.currentTargetStatus", "*", 1,
                                                         time_cut=date_eoit.isot)
focus_offset_sum_df = await client.select_top_n("lsst.sal.ATAOS.logevent_focusOffsetSummary", "*", 1,
                                                time_cut=date_start_int.isot)
atmcs_target_df = await client.select_top_n("lsst.sal.ATMCS.logevent_target", "*", 1,
                                            time_cut=date_start_int.isot)
atmcs_m3selected_df = await client.select_top_n("lsst.sal.ATMCS.logevent_m3PortSelected", "selected", 1,
                                                time_cut=date_start_int.isot)
atspec_settings_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_settingsAppliedValues", "*", 1,
                                               time_cut=date_start_int.isot)

if obs_date >= 20210318:
    params = "azimuthCalculatedAngle0,declination0,elevationCalculatedAngle0,ra0,skyAngle0"
    mount_positions_start_df = await client.select_top_n("lsst.sal.ATPtg.mountPositions", params, 1,
                                                        time_cut=date_start_int.isot)
    mount_positions_end_df = await client.select_top_n("lsst.sal.ATPtg.mountPositions", params, 1,
                                                       time_cut=date_eoit.isot)
else:
    mount_positions_start_df = await client.select_top_n("lsst.sal.ATPtg.mount_positions", "*", 1,
                                                        time_cut=date_start_int.isot)
    mount_positions_end_df = await client.select_top_n("lsst.sal.ATPtg.mount_positions", "*", 1,
                                                       time_cut=date_eoit.isot)

In [ ]:
if header["IMGTYPE"] in ["OBJECT", "ENGTEST"]:
    check("OBJECT", header, current_target_df["targetName"][0])
    check_float("RA", header, np.degrees(current_target_df["ra"])[0])
    check_float("DEC", header, np.degrees(current_target_df["declination"])[0])
    check_float("ROTPA", header, mount_positions_start_df["skyAngle0"][0])
    check("ROTCOORD", header, "sky")
    check_float("RASTART", header, mount_positions_start_df["ra0"][0])
    check_float("RAEND", header, mount_positions_end_df["ra0"][0])
    check_near("RA", "RASTART", header, 0.5)
    check_near("RA", "RAEND", header, 0.5)
    check_float("DECSTART", header, mount_positions_start_df["declination0"][0])
    check_float("DECEND", header, mount_positions_end_df["declination0"][0])
    check_near("DEC", "DECSTART", header, 0.1)
    check_near("DEC", "DECEND", header, 0.1)
    check_float("HASTART", header, current_target_status_start_df["ha"][0])
    check_float("HAEND", header, current_target_status_end_df["ha"][0])
    check_float("AMSTART", header, current_target_status_start_df["airmass"][0])
    check_float("AMEND", header, current_target_status_end_df["airmass"][0])
else:
    print(f"No check on RA/DEC for {header['IMGTYPE']}")
check_float("AZSTART", header, mount_positions_start_df["azimuthCalculatedAngle0"][0])
check_float("AZEND", header, mount_positions_end_df["azimuthCalculatedAngle0"][0])
check_float("ELSTART", header, mount_positions_start_df["elevationCalculatedAngle0"][0])
check_float("ELEND", header, mount_positions_end_df["elevationCalculatedAngle0"][0])
check_float("FOCUSZ", header, focus_offset_sum_df["userApplied"][0])
check("TRACKSYS", header, atmcs_target_df["tracksys"][0])
check("RADESYS", header, atmcs_target_df["radesys"][0])
check("INSTPORT", header, atspec_settings_df["instrumentPort"][0])
check("ATM3PORT", header, atmcs_m3selected_df["selected"][0])
#check_not_empty("ATM3PORT", header)

In [ ]:
sim_atmcs_df = await client.select_top_n("lsst.sal.ATMCS.logevent_simulationMode", "mode", 1,
                                         time_cut=date_start_int.isot)
sim_atpneumatics_df = await client.select_top_n("lsst.sal.ATPneumatics.logevent_simulationMode", "mode", 1,
                                                time_cut=date_start_int.isot)
sim_athexapod_df = await client.select_top_n("lsst.sal.ATHexapod.logevent_simulationMode", "mode", 1,
                                             time_cut=date_start_int.isot)
sim_atspectrograph_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_simulationMode", "mode", 1,
                                                  time_cut=date_start_int.isot)
sim_atdome_df = await client.select_top_n("lsst.sal.ATDome.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)

In [ ]:
check("SIMULATE ATMCS", header, sim_atmcs_df["mode"][0])
check("SIMULATE ATPNEUMATICS", header, sim_atpneumatics_df["mode"][0])
check("SIMULATE ATHEXAPOD", header, sim_athexapod_df["mode"][0])
check("SIMULATE ATSPECTROGRAPH", header, sim_atspectrograph_df["mode"][0])
check("SIMULATE ATDOME", header, sim_atdome_df["mode"][0])

In [ ]:
filter_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedFilterPosition", "*", 1,
                                          time_cut=date_start_int.isot)
grating_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedDisperserPosition", "*", 1,
                                           time_cut=date_start_int.isot)
linear_stage_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedLinearStagePosition",
                                                "*", 1,
                                                time_cut=date_start_int.isot)
client.influx_client.output = "json"
filter_pos_json = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedFilterPosition", "*", 1,
                                            time_cut=date_start_int.isot)
grating_pos_json = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedDisperserPosition", "*", 1,
                                             time_cut=date_start_int.isot)
client.influx_client.output = "dataframe"

In [ ]:
check("FILTER", header, get_from_json("name", filter_pos_json))
check("FILTBAND", header, filter_pos_df["band"][0])
check("FILTSLOT", header, filter_pos_df["slot"][0])
check("GRATING", header, get_from_json("name", grating_pos_json))
check("GRATBAND", header, grating_pos_df["band"][0])
check("GRATSLOT", header, grating_pos_df["slot"][0])
check_float("LINSPOS", header, linear_stage_pos_df["position"][0])

In [ ]:
focal_plane_info_df = await client.select_top_n("lsst.sal.ATCamera.logevent_focalPlaneSummaryInfo", "*", 1,
                                                time_cut=date_start_int.isot)
fp_ccd_df = await client.select_top_n("lsst.sal.ATCamera.focal_plane_Ccd", "*", 1,
                                      time_cut=date_start_int.isot)
vacuum_df = await client.select_top_n("lsst.sal.ATCamera.vacuum", "*", 1,
                                      time_cut=date_start_int.isot)

In [ ]:
check("CCD_MANU", header, focal_plane_info_df["ccdManufacturer"][0])
check("CCD_TYPE", header, focal_plane_info_df["ccdType"][0])
check("CCD_SERN", header, focal_plane_info_df["ccdManSerNum"][0])
check("LSST_NUM", header, focal_plane_info_df["ccdLSSTName"][0])
check("CCDSLOT", header, focal_plane_info_df["ccdSlot"][0])
check("RAFTBAY", header, focal_plane_info_df["raftBay"][0])
check("SEQCKSUM", header, focal_plane_info_df["sequencerChecksum"][0])
check("SEQNAME", header, focal_plane_info_df["sequencerKey"][0])
check("REBNAME", header, focal_plane_info_df["rebLSSTName"][0])
check("CONTNUM", header, focal_plane_info_df["rebSerialNumber"][0])
check_float("CCDTEMP", header, fp_ccd_df["temp0"][0])
check_float("TEMP_SET", header, focal_plane_info_df["ccdTempSetPoint"][0])
check("IMAGETAG", header, end_image_tel["imageTag"][0])

In [ ]:
weather_df = await client.select_top_n("lsst.sal.WeatherStation.weather", "*", 1,
                                       time_cut=date_start_int.isot)
wind_speed_df = await client.select_top_n("lsst.sal.WeatherStation.windSpeed", "*", 1,
                                          time_cut=date_start_int.isot)
wind_dir_df = await client.select_top_n("lsst.sal.WeatherStation.windDirection", "*", 1,
                                        time_cut=date_start_int.isot)
seeing_df = await client.select_top_n("lsst.sal.DIMM.logevent_dimmMeasurement", "*", 1,
                                      time_cut=date_start_int.isot)
dome_pos_df = await client.select_top_n("lsst.sal.ATDome.position", "*", 1,
                                        time_cut=date_start_int.isot)

In [ ]:
check_float("AIRTEMP", header, weather_df["ambient_temp"][0])
check_float("PRESSURE", header, weather_df["pressure"][0])
check_float("HUMIDITY", header, weather_df["humidity"][0])
check_float("WINDSPD", header, wind_speed_df["value"][0])
check_float("WINDDIR", header, wind_dir_df["value"][0])
check_float("SEEING", header, seeing_df["fwhm"][0])
check_float("DOMEAZ", header, dome_pos_df["azimuthPosition"][0])
check_float("SHUTLOWR", header, dome_pos_df["dropoutDoorOpeningPercentage"][0])
check_float("SHUTUPPR", header, dome_pos_df["mainDoorOpeningPercentage"][0])
check("FACILITY", header, "Vera C. Rubin Observatory")

In [ ]:
check_not_empty("DATE", header)
check_not_empty("MJD-OBS", header)
check_not_empty("MJD-BEG", header)
check_not_empty("MJD-END", header)
check("BUNIT", header, "adu")
check("TELESCOP", header, "LSST AuxTelescope")
check("INSTRUME", header, "LATISS")
check_float("OBS-LONG", header, -70.749417)
check_float("OBS-LAT", header, -30.244639)
check_float("OBS-ELEV", header, 2663.0)
check_float("OBSGEO-X", header, 1818938.94)
check_float("OBSGEO-Y", header, -5208470.95)
check_float("OBSGEO-Z", header, -3195172.08)
check_not_empty("FILENAME", header)
check_not_empty("HEADVER", header)
check_not_empty("TSTAND", header)
check_not_empty("CHECKSUM", header)
check_not_empty("DATASUM", header)
check("TIMESYS", header, "TAI")

In [ ]:
check("XTENSION", header, "IMAGE")
check("BITPIX", header, 32)
check("INHERIT", header, "T")
check("PCOUNT", header, 0)
check("GCOUNT", header, 1)
check_not_empty("EXTNAME", header)
check_not_empty("DATASEC", header)
check_not_empty("DETSEC", header)
check_not_empty("DTV1", header)
check_not_empty("DTV2", header)
check_not_empty("DTM1_1", header)
check_not_empty("DTM1_2", header)
check_not_empty("DTM2_1", header)
check_not_empty("DTM2_2", header)